# Import graspi_igraph

Ensure that you are only using python versions higher than 3.7
- Python version can be checked by running *python --version* in the terminal
- it can also be checked on notebook via *!pip --version* in the code section of the notebook

## Visualization function call
---
change the file name if you want to visualize other result


In [22]:
!pip install pandas
!pip install matplotlib
!pip install igraph



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip



## Testing Code
---
Doing different igraph file test & generating CSV file(for comparing performance)

### Testing :: src/graph.py (PERIODICITY = False)

In [7]:
import sys
import os

current_dir = os.getcwd()
src_path = os.path.abspath(os.path.join(current_dir, "../src"))
sys.path.append(src_path)

import time
import matplotlib.pyplot as plt
import tracemalloc
import importlib

from py_graspi import descriptors as ds
from py_graspi import graph as ig


importlib.reload(ig)  # reload
importlib.reload(ds)  # reload


current_dir = os.getcwd()

# data_path = f"../data/2D-testFile/" # Datafile path
data_path = f"../data/data/" # 33 morphology

descriptors_path = f"../data/descriptors_from_C++/" # C++ results path
result_path = f"./test_results/" # test results will be saved in this path
test_files = [os.path.splitext(file)[0] for file in os.listdir(data_path)]
epsilon=1e-5

times = []
mems = []
time_mem_stats = {}

for test_file in test_files:
    import time
    tracemalloc.start()
    graph_start = time.time()
    g_data = ig.generateGraph(data_path+test_file+".txt")    
    mem_graph = tracemalloc.get_traced_memory()
    graph_end = time.time()
    tracemalloc.stop()
    graph_mem = mem_graph[1]-mem_graph[0]  
    
    tracemalloc.start()
    desc_start = time.time()
    stats = ds.compute_descriptors(g_data, test_file+".txt", 1)
    mem_desc = tracemalloc.get_traced_memory()
    desc_end = time.time()
    tracemalloc.stop()
    descriptor_time = desc_end - desc_start
    descriptor_mem = mem_desc[1]-mem_desc[0]
    #ig.visual2D(g, 'graph')

    print(f"{test_file} Results")

    try:
        with open(descriptors_path + "descriptors." + test_file + ".log") as f:
            for line in f:
                stat = line.strip().split(" ")
                try:
                    if abs(stats.get(stat[0], -1) - float(stat[1])) < epsilon:
                        print(f"{stat[0]} passed")
                    elif stats.get(stat[0], -1) != -1 and stats.get(stat[0], -1) != int(stat[1]):
                        print(f"{stat[0]} failed - {stats.get(stat[0])} is not the same as expected {stat[1]}")
                except ValueError:
                    if abs(stats.get(stat[0], -1) - float(stat[1])) < epsilon:
                        print(f"{stat[0]} passed")
                    elif stats.get(stat[0], -1) != -1 and stats.get(stat[0], -1) != float(stat[1]):
                        print(f"{stat[0]} failed - {stats.get(stat[0])} is not the same as expected {stat[1]}")

    except FileNotFoundError:
        print(f"File not found: {descriptors_path + 'descriptors.' + test_file + '.log'}")
        for key in stats:
            print(f"new {key}: {stats[key]}")
            print(f"prev {key}: {stats[key]}")                     


    times.append(descriptor_time)
    mems.append(descriptor_mem)

    graph_time = graph_end-graph_start
    print(f"Total time to calculate graph: {graph_time} second(s)")
    print(f"Total time to calculate descriptors: {descriptor_time} second(s)")
    print(f"Peak memory usage for graph generation: {graph_mem} bytes")
    print(f"Peak memory usage for descriptor calculation: {descriptor_mem} bytes")
    print(stats)
    print("")
    time_mem_stats[test_file] = {"graph_time": graph_time, "descriptor_time": descriptor_time,  "graph_mem":graph_mem, "descriptor_mem": descriptor_mem}



data_0.5_2.2_001900 Results
STAT_n passed
STAT_e passed
STAT_n_D passed
STAT_n_A passed
STAT_CC_D passed
STAT_CC_A passed
STAT_CC_D_An passed
STAT_CC_A_Ca passed
ABS_wf_D failed - 0.31148777712164943 is not the same as expected 0.308388
ABS_f_D passed
DISS_wf10_D failed - 0.21415371456093757 is not the same as expected 0.2166
DISS_f10_D failed - 0.4223397426099716 is not the same as expected 0.423287
CT_f_e_conn passed
CT_f_conn_D_An passed
CT_f_conn_A_Ca passed
CT_e_conn passed
CT_e_D_An passed
CT_e_A_Ca passed
CT_f_D_tort1 passed
CT_f_A_tort1 passed
CT_n_D_adj_An failed - 512 is not the same as expected 513
CT_n_A_adj_Ca failed - 512 is not the same as expected 513
Total time to calculate graph: 33.13182806968689 second(s)
Total time to calculate descriptors: 16.038447856903076 second(s)
Peak memory usage for graph generation: 59118577 bytes
Peak memory usage for descriptor calculation: 84474754 bytes
{'STAT_n': 65536, 'STAT_e': 1612, 'STAT_n_D': 32713, 'STAT_n_A': 32823, 'STAT_CC_D'

KeyboardInterrupt: 

# Helper Functions
---


In [19]:
###########################################################################################
# function that makes csv file of dictionary 
###########################################################################################
import csv

def save_dict_to_csv(data_dict, file_name):
    """
    Saves a given dictionary as a CSV file.
    :param data_dict: Dictionary to be saved (keys are filenames, values are nested dictionaries with data)
    :param file_name: Name of the CSV file to be saved (e.g., 'output.csv')
    """
    # Extract headers from the first key
    headers = ["Test File"]  # First column is the file name
    if data_dict:
        sample_key = next(iter(data_dict))
        headers.extend(data_dict[sample_key].keys())
    
    # Write to CSV file
    with open(file_name, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(headers)  # Write headers
        
        for key, values in data_dict.items():
            row = [key] + [values.get(h, None) for h in headers[1:]]
            writer.writerow(row)  # Write data rows
    
    print(f"CSV file '{file_name}' has been successfully saved.")


In [ ]:
'''run this cell only when you want to save the performance metric to csv'''

# change file name 
save_dict_to_csv(time_mem_stats, "performance/stats_new_2D.csv")

CSV file 'performance/stats_new_2D.csv' has been successfully saved.


### Testing :: src/graph.py (PERIODICITY = True)

In [ ]:
import sys
import os

current_dir = os.getcwd()
src_path = os.path.abspath(os.path.join(current_dir, "../src"))
sys.path.append(src_path)

import time
import matplotlib.pyplot as plt
import tracemalloc
import importlib

from py_graspi import descriptors as ds
from py_graspi import graph as ig


importlib.reload(ig)  # reload
importlib.reload(ds)  # reload


current_dir = os.getcwd()

# data_path = f"../data/2D-testFile/" # Datafile path
data_path = f"../data/data/" # 33 morphology

descriptors_path = f"../data/descriptors/" # C++ results path
result_path = f"../test_results/" # test results will be saved in this path
test_files = [os.path.splitext(file)[0] for file in os.listdir(data_path)]
epsilon=1e-5

times = []
mems = []
time_mem_stats = {}

for test_file in test_files:
    import time
    tracemalloc.start()
    graph_start = time.time()
    g_data = ig.generateGraph(data_path+test_file+".txt", True)    
    mem_graph = tracemalloc.get_traced_memory()
    graph_end = time.time()
    tracemalloc.stop()
    graph_mem = mem_graph[1]-mem_graph[0]  
    
    tracemalloc.start()
    desc_start = time.time()
    stats = ds.compute_descriptors(g_data, test_file+".txt", 1)
    mem_desc = tracemalloc.get_traced_memory()
    desc_end = time.time()
    tracemalloc.stop()
    descriptor_time = desc_end - desc_start
    descriptor_mem = mem_desc[1]-mem_desc[0]
    #ig.visual2D(g, 'graph')

    print(f"{test_file} Results")

    try:
        with open(descriptors_path + "descriptors." + test_file + ".log") as f:
            for line in f:
                stat = line.strip().split(" ")
                try:
                    if abs(stats.get(stat[0], -1) - float(stat[1])) < epsilon:
                        print(f"{stat[0]} passed")
                    elif stats.get(stat[0], -1) != -1 and stats.get(stat[0], -1) != int(stat[1]):
                        print(f"{stat[0]} failed - {stats.get(stat[0])} is not the same as expected {stat[1]}")
                except ValueError:
                    if abs(stats.get(stat[0], -1) - float(stat[1])) < epsilon:
                        print(f"{stat[0]} passed")
                    elif stats.get(stat[0], -1) != -1 and stats.get(stat[0], -1) != float(stat[1]):
                        print(f"{stat[0]} failed - {stats.get(stat[0])} is not the same as expected {stat[1]}")

    except FileNotFoundError:
        print(f"File not found: {descriptors_path + 'descriptors.' + test_file + '.log'}")
        for key in stats:
            print(f"new {key}: {stats[key]}")
            print(f"prev {key}: {stats[key]}")                     


    times.append(descriptor_time)
    mems.append(descriptor_mem)

    graph_time = graph_end-graph_start
    print(f"Total time to calculate graph: {graph_time} second(s)")
    print(f"Total time to calculate descriptors: {descriptor_time} second(s)")
    print(f"Peak memory usage for graph generation: {graph_mem} bytes")
    print(f"Peak memory usage for descriptor calculation: {descriptor_mem} bytes")
    print(stats)
    print("")
    time_mem_stats[test_file] = {"graph_time": graph_time, "descriptor_time": descriptor_time,  "graph_mem":graph_mem, "descriptor_mem": descriptor_mem}



FileNotFoundError: [Errno 2] No such file or directory: './results/data_0.5_2.2_001900_TortuosityBlackToRed.txt'

In [ ]:
# change file name 
save_dict_to_csv(time_mem_stats, "performance/stats_new_2D_periodicity.csv")

In [ ]:
import sys
import os
sys.path.append(os.path.abspath('..'))
import src.graph as ig
import src.descriptors as ds
import time
import matplotlib.pyplot as plt
import tracemalloc
import src.graph_data_class as gc
import importlib

importlib.reload(ig)  # reload the module (apply modification)
importlib.reload(ds)  # reload the module (apply modification)


current_dir = os.getcwd()
data_path = f"../data/3phase/data/"
descriptors_path = f"../data/descriptors_from_C++/"
result_path = f"./results/"
test_files = [os.path.splitext(file)[0] for file in os.listdir(data_path)]
epsilon=1e-5

times = []
mems = []
time_mem_stats = {}

for test_file in test_files:
    import time
    tracemalloc.start()
    graph_start = time.time()
    g_data = ig.generateGraph(data_path+test_file+".txt")    
    mem_graph = tracemalloc.get_traced_memory()
    graph_end = time.time()
    tracemalloc.stop()
    graph_mem = mem_graph[1]-mem_graph[0]  
    
    if g_data.black_green == 0:
        print(f"{test_file} Results")
        print("STAT_e : 0")
        print("cannot calculate other descriptors")
        print("")
        continue
    tracemalloc.start()
    desc_start = time.time()
    stats = ds.descriptors(g_data, test_file+".txt")
    mem_desc = tracemalloc.get_traced_memory()
    desc_end = time.time()
    tracemalloc.stop()
    descriptor_time = desc_end - desc_start
    descriptor_mem = mem_desc[1]-mem_desc[0]
    #ig.visual2D(g, 'graph')

    print(f"{test_file} Results")

    try:
        with open(descriptors_path + "descriptors." + test_file + ".log") as f:
            for line in f:
                stat = line.strip().split(" ")
                try:
                    # if stats.get(stat[0], -1) == int(stat[1]):
                    if abs(stats.get(stat[0], -1) - float(stat[1])) < epsilon:
                        print(f"{stat[0]} passed")
                    elif stats.get(stat[0], -1) != -1 and stats.get(stat[0], -1) != int(stat[1]):
                        print(f"{stat[0]} failed - {stats.get(stat[0])} is not the same as expected {stat[1]}")
                except ValueError:
                    if abs(stats.get(stat[0], -1) - float(stat[1])) < epsilon:
                        print(f"{stat[0]} passed")
                    elif stats.get(stat[0], -1) != -1 and stats.get(stat[0], -1) != float(stat[1]):
                        print(f"{stat[0]} failed - {stats.get(stat[0])} is not the same as expected {stat[1]}")

    except FileNotFoundError:
        print(f"File not found: {descriptors_path + 'descriptors.' + test_file + '.log'}")
        for key in stats:
            print(f"{key}: {stats[key]}")

    times.append(descriptor_time)
    mems.append(descriptor_mem)

    graph_time = graph_end-graph_start
    print(f"Total time to calculate graph: {graph_time} second(s)")
    print(f"Total time to calculate descriptors: {descriptor_time} second(s)")
    print(f"Peak memory usage for graph generation: {graph_mem} bytes")
    print(f"Peak memory usage for descriptor calculation: {descriptor_mem} bytes")
    print(stats)
    print("")
    time_mem_stats[test_file] = {"graph_time": graph_time, "descriptor_time": descriptor_time,  "graph_mem":graph_mem, "descriptor_mem": descriptor_mem}



dataRot Results
STAT_e : 0
cannot calculate other descriptors

data_10_10 Results
File not found: ../data/descriptors/descriptors.data_10_10.log
STAT_n: 67
STAT_e: 19
STAT_n_D: 26
STAT_n_A: 41
STAT_CC_D: 3
STAT_CC_A: 4
STAT_CC_D_An: 2
STAT_CC_A_Ca: 2
ABS_wf_D: 0.38405732149656174
ABS_f_D: 0.3880597014925373
DISS_f10_D: 1.0
DISS_wf10_D: 0.8786200179624305
CT_f_e_conn: 0.5789473684210527
CT_f_conn_D_An: 0.9615384615384616
CT_f_conn_A_Ca: 0.9512195121951219
CT_e_conn: 11
CT_e_D_An: 21
CT_e_A_Ca: 12
CT_n_D_adj_An: 4
CT_n_A_adj_Ca: 6
CT_f_D_tort1: 0.96
CT_f_A_tort1: 0.7692307692307693
Total time to calculate graph: 0.0362088680267334 second(s)
Total time to calculate descriptors: 0.021864891052246094 second(s)
Peak memory usage for graph generation: 65621 bytes
Peak memory usage for descriptor calculation: 56551 bytes
{'STAT_n': 67, 'STAT_e': 19, 'STAT_n_D': 26, 'STAT_n_A': 41, 'STAT_CC_D': 3, 'STAT_CC_A': 4, 'STAT_CC_D_An': 2, 'STAT_CC_A_Ca': 2, 'ABS_wf_D': 0.38405732149656174, 'ABS_f_D': 